In [1]:
# This program serves to isolate and store the balrog galaxies that pass quality cuts.

In [2]:
import numpy as np
import fitsio
import astropy.io.fits as fits
from astropy.table import Table

In [3]:
deepFile1 = '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000001.fits'
deepFile2 = '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000002.fits'
deepFile3 = '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000003.fits'
deepFile4 = '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000004.fits'
deepFile5 = '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000005.fits'
deepCols  = ['KNN_CLASS', 'ID']
deepData1 = fitsio.read(deepFile1, columns = deepCols)
deepData2 = fitsio.read(deepFile2, columns = deepCols)
deepData3 = fitsio.read(deepFile3, columns = deepCols)
deepData4 = fitsio.read(deepFile4, columns = deepCols)
deepData5 = fitsio.read(deepFile5, columns = deepCols)

deepID = []
deepID.extend(list(deepData1['ID']))
deepID.extend(deepData2['ID'])
deepID.extend(deepData3['ID'])
deepID.extend(deepData4['ID'])
deepID.extend(deepData5['ID'])
deepID = np.array(deepID)

deepClass = []
deepClass.extend(deepData1['KNN_CLASS'])
deepClass.extend(deepData2['KNN_CLASS'])
deepClass.extend(deepData3['KNN_CLASS'])
deepClass.extend(deepData4['KNN_CLASS'])
deepClass.extend(deepData5['KNN_CLASS'])
deepClass = np.array(deepClass)

In [4]:
minID = np.min(deepID)
deepGalID = np.zeros(np.max(deepID) - minID + 1)
deepGalID[deepID - minID] = deepClass # To access the class at an id, do id - minID for indexing.

In [5]:
# ssiFile = '/hdfs/bechtol/balrog/y3/balrog_matched_catalog_sof_y3-merged_v1.2.fits'
# ssiCols = ['true_id', 'bal_id', true_bdf_mag_deredden', 'meas_EXTENDED_CLASS_SOF', 'true_ra', 'true_dec']
# ssiData = fitsio.read(ssiFile, columnns = ssiCols)

In [6]:
balrData = fits.open("/hdfs/bechtol/balrog/y3/balrog_detection_catalog_sof_y3-merged_v1.2.fits")[1].data

In [7]:
ssiID = balrData['true_id']
RA = balrData['true_ra']
DEC = balrData['true_dec']
CLASS = balrData['meas_EXTENDED_CLASS_SOF']
GMAG = balrData['true_bdf_mag_deredden'][:,0]

RMAG = balrData['true_bdf_mag_deredden'][:,1]
DETECTED = balrData['detected']
FOREGROUND = balrData['flags_foreground']
BADREGIONS = balrData['flags_badregions']
FOOTPRINT = balrData['flags_footprint']
ARCSECONDS = balrData['match_flag_1.5_asec']

cutIndices = np.where((DETECTED == 1) &
                      (FOREGROUND == 0) & 
                      (BADREGIONS < 2) & 
                      (FOOTPRINT == 1) & 
                      (ARCSECONDS < 2) &
                      (CLASS >= 0) &
                      (GMAG - RMAG >= -0.3) &
                      (GMAG - RMAG <= 1))[0]

ssiID = ssiID[cutIndices]
RA = RA[cutIndices]
DEC = DEC[cutIndices]
CLASS = CLASS[cutIndices]
GMAG = GMAG[cutIndices]

ssiTrueClass = deepGalID[ssiID - minID]
useInds = np.where(ssiTrueClass == 1)[0]

RA = RA[useInds]
DEC = DEC[useInds]
CLASS = CLASS[useInds]
GMAG = GMAG[useInds]

In [8]:
file = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/GalaxyContamination/'

In [9]:
my_table = Table()
my_table['RA'] = RA
my_table['DEC'] = DEC
my_table['GMAG'] = GMAG
my_table['CLASS'] = CLASS
my_table.write(file + 'Blue_Galaxies.fits', overwrite = True)

In [12]:
# This leaves about 5 million objects to perform analysis on.